In [1]:
from plyfile import PlyData
import numpy as np
import open3d

# Load the binary .ply
pcd = open3d.io.read_point_cloud("/Users/dragos/Licenta/data/car1_resized/test.ply")
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

print(points.shape)
# print(colors)


ply = PlyData.read("/Users/dragos/Licenta/data/car1_resized/test.ply")
vertex = ply['vertex'].data

xyz = np.stack([vertex['x'], vertex['y'], vertex['z']], axis=-1)

# print(xyz)

rgb = np.stack([vertex['f_dc_0'], vertex['f_dc_1'], vertex['f_dc_2']], axis=-1)

print(rgb.shape)

# Optional: Clamp or normalize if needed
# rgb = np.clip(rgb, 0, 1)  # assuming values are already in [0,1] range

# Convert to Open3D PointCloud
# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(xyz)
# pcd.colors = o3d.utility.Vector3dVector(rgb)
# pcd.colors = open3d.utility.Vector3dVector(rgb)

# open3d.visualization.draw_geometries([pcd])

(13312, 3)
(13312, 3)


In [1]:
import open3d as o3d
import numpy as np
from plyfile import PlyData

# Load your point cloud
pcd = o3d.io.read_point_cloud('/Users/dragos/Licenta/data/car1_resized/test.ply')

ply = PlyData.read("/Users/dragos/Licenta/data/car1_resized/test.ply")
vertex = ply['vertex'].data
rgb = np.stack([vertex['f_dc_0'], vertex['f_dc_1'], vertex['f_dc_2']], axis=-1)
# rgb = np.clip(rgb, 0, 1)
# rgb = rgb.astype(np.float64)
# print(rgb.shape, rgb.min(), rgb.max())
print(rgb)

# Get the points
points = np.asarray(pcd.points)
print(f"Number of points: {len(points)}")

# Check if colors exist
if not pcd.has_colors():
    print("Point cloud has no colors, generating new ones...")
    
    # Create a new color array - here are different options:
    
    # Option 1: Single color for all points (e.g., white)
    # colors = np.ones((len(points), 3)) * 0.8  # Light gray color
    
    # Option 2: Random colors
    colors = np.random.random((len(points), 3))
    # print(colors.shape, colors.min(), colors.max())
    print(colors)
    
    # Option 3: Color based on position (normalized coordinates)
    # Normalize x, y, z to [0,1] for RGB channels
    # normalized_points = points - np.min(points, axis=0)
    # normalized_points = normalized_points / np.max(normalized_points, axis=0)
    # colors = normalized_points
    
    # Option 4: Color based on height (z-coordinate) 
    # z_min, z_max = np.min(points[:, 2]), np.max(points[:, 2])
    # z_normalized = (points[:, 2] - z_min) / (z_max - z_min)
    # colors = np.zeros((len(points), 3))
    # colors[:, 0] = 1 - z_normalized  # Red channel (more red at bottom)
    # colors[:, 1] = z_normalized      # Green channel (more green at top)
    # colors[:, 2] = z_normalized * 0.5  # Blue channel
    
    # Assign the new colors to the point cloud
    pcd.colors = o3d.utility.Vector3dVector(colors)
else:
    print("Point cloud already has colors")
    colors = np.asarray(pcd.colors)
    print(f"Color shape: {colors.shape}")

if np.any(np.isnan(rgb)) or np.any(np.isinf(rgb)):
    print("Found NaNs or Infs in RGB!")
else:
    print("No NaNs or Infs in RGB!")

print(f"RGB data type: {rgb.dtype}")
print(f"Colors data type: {colors.dtype}")

# o3d.visualization.draw_plotly([pcd])
# o3d.io.write_point_cloud('/Users/dragos/Licenta/data/car1_resized/colored_output.ply', pcd)

[[ 0.72727543  0.98554987  1.3920512 ]
 [ 1.6852447   1.4741848   2.0681632 ]
 [ 1.0134789   1.0947521   1.3718374 ]
 ...
 [-1.7813747  -1.7790794  -1.5375466 ]
 [-1.7856042  -1.7891855  -1.8102732 ]
 [-0.77595335 -0.99894506 -0.8963592 ]]
Number of points: 13312
Point cloud has no colors, generating new ones...
[[0.99416362 0.309212   0.72792861]
 [0.99520374 0.26080273 0.95807812]
 [0.611328   0.04782519 0.70120763]
 ...
 [0.96375617 0.2436263  0.13275916]
 [0.39023853 0.39316098 0.62836858]
 [0.62257187 0.82284788 0.6246812 ]]
No NaNs or Infs in RGB!
RGB data type: float32
Colors data type: float64


In [2]:
from plyfile import PlyElement, PlyData
import numpy as np

new_vertex_data = []

for i in range(len(vertex)):
    v = vertex[i]

    # Replace only f_dc_* with updated RGB
    new_vertex = (
        v['x'], v['y'], v['z'],
        v['nx'], v['ny'], v['nz'],
        rgb[i, 0], rgb[i, 1], rgb[i, 2],  # updated colors
        *[v[f"f_rest_{j}"] for j in range(45)],  # f_rest_0 to f_rest_44
        v['opacity'],
        v['scale_0'], v['scale_1'], v['scale_2'],
        v['rot_0'], v['rot_1'], v['rot_2'], v['rot_3'],
    )

    new_vertex_data.append(new_vertex)

# Define the data structure
vertex_dtype = [
    ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
    ('nx', 'f4'), ('ny', 'f4'), ('nz', 'f4'),
    ('f_dc_0', 'f4'), ('f_dc_1', 'f4'), ('f_dc_2', 'f4'),
]

# Add all f_rest_* fields
vertex_dtype += [(f"f_rest_{i}", 'f4') for i in range(45)]

# Remaining fields
vertex_dtype += [
    ('opacity', 'f4'),
    ('scale_0', 'f4'), ('scale_1', 'f4'), ('scale_2', 'f4'),
    ('rot_0', 'f4'), ('rot_1', 'f4'), ('rot_2', 'f4'), ('rot_3', 'f4')
]

# Create numpy array
vertex_array = np.array(new_vertex_data, dtype=vertex_dtype)

# Create PlyElement and PlyData
ply_el = PlyElement.describe(vertex_array, 'vertex')
ply_data = PlyData([ply_el], text=False)

# Save to disk
ply_data.write('/Users/dragos/Licenta/data/car1_resized/colored_output_custom.ply')